# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [3]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
#     print(source_vocab_to_int)
#     print([word for word in source_text.split()])
    source_int = [[source_vocab_to_int[word] for word in se.split()] for se in source_text.split('\n')]
    target_int = [[target_vocab_to_int[word] for word in se.split()+['<EOS>']] for se in target_text.split('\n')]
    return source_int, target_int

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.3.0


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: No GPU found. Please use a GPU to train your neural network.
  from ipykernel import kernelapp as app


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [7]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    input_data = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    lr = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob= tf.placeholder(tf.float32,name="keep_prob")

    target_sequence_length = tf.placeholder(tf.int32, (None,), name='target_sequence_length')
    max_target_sequence_length = tf.reduce_max(target_sequence_length, name='max_target_len')
    source_sequence_length = tf.placeholder(tf.int32, (None,), name='source_sequence_length')
    
    return input_data, targets, lr, keep_prob,target_sequence_length, max_target_sequence_length, source_sequence_length



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'assert_rank_2/Assert/Assert' type=Assert>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
['File "/opt/conda/lib/python3.6/runpy.py", line 193, in _run_module_as_main\n    "__main__", mod_spec)', 'File "/opt/conda/lib/python3.6/runpy.py", line 85, in _run_code\n    exec(code, run_globals)', 'File "/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>\n    app.launch_new_instance()', 'File "/opt/conda/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance\n    app.start()', 'File "/opt/conda/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start\n    ioloop.IOLoop.instance().start()', 'File "/opt/conda/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 177, in start\n    super(ZMQIOLoop, self).start()', 'F

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [8]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']), ending], 1)

    return dec_input

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [9]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # TODO: Implement Function
    embedding=tf.contrib.layers.embed_sequence(rnn_inputs, source_vocab_size, encoding_embedding_size)
    def make_cell(rnn_size):
        cell = tf.contrib.rnn.LSTMCell(rnn_size, initializer=tf.random_uniform_initializer(-.1,0.1,seed=3))
        dropout=tf.contrib.rnn.DropoutWrapper(cell,input_keep_prob=keep_prob)
        return dropout
    enc_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
    enc_output, enc_state = tf.nn.dynamic_rnn(enc_cell, embedding, sequence_length=source_sequence_length, dtype=tf.float32)
    return enc_output, enc_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [10]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    
    # 4. Set up a training decoder and an inference decoder
    # Training Decoder
    with tf.variable_scope("decode"):

        # Helper for the training process. Used by BasicDecoder to read inputs.
        training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                            sequence_length=target_sequence_length,
                                                            time_major=False)
        
        
        # Basic decoder
        training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                           training_helper,
                                                           encoder_state,
                                                           output_layer) 
        
        # Perform dynamic decoding using the decoder
        training_decoder_output = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                                    impute_finished=True,
                                                                    maximum_iterations=max_summary_length)[0]
    

    return training_decoder_output

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [11]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function
    # 5. Inference Decoder
    # Reuses the same parameters trained by the training process
    with tf.variable_scope("decode", reuse=None):
        start_tokens = tf.tile(tf.constant([start_of_sequence_id], dtype=tf.int32), [batch_size], name='start_tokens')

        # Helper for the inference process.
        inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings,
                                                                start_tokens,
                                                                end_of_sequence_id)

        # Basic decoder
        inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                        inference_helper,
                                                        encoder_state,
                                                        output_layer)
        
        # Perform dynamic decoding using the decoder
        inference_decoder_output = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                            impute_finished=True,
                                                            maximum_iterations=max_target_sequence_length)[0]
         
    return inference_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [16]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    # 1. Decoder Embedding
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)

    # 2. Construct the decoder cell
    def make_cell(rnn_size, i_layer):
        with tf.variable_scope('decoder_{}'.format(i_layer)):
            lstm = tf.contrib.rnn.LSTMCell(rnn_size,
                                               initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            dec_cell = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob=keep_prob)
            return dec_cell

    dec_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size, i) for i in range(num_layers)])
     
    # 3. Dense layer to translate the decoder's output at each time 
    # step into a choice from the target vocabulary
    with tf.variable_scope("decoding") as decoding_scope:
        output_layer = Dense(target_vocab_size,
                             kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1),
                             name='output_layer')

        output= decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, 
                                     max_target_sequence_length, output_layer, keep_prob)

        infer = decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, target_vocab_to_int['<GO>'],
                                     target_vocab_to_int['<EOS>'], max_target_sequence_length, target_vocab_size, 
                                     output_layer, batch_size, keep_prob)

    return output, infer


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

(64, ?, 1000)
[64, None, 1000]
Tests Passed


In [ ]:
%load_ext autoreload
%autoreload 2

### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [18]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
     # Pass the input data through the encoder. We'll ignore the encoder output, but use the state
    _, enc_state = encoding_layer(input_data, 
                                  rnn_size, 
                                  num_layers, 
                                  source_sequence_length=source_sequence_length,
                                  source_vocab_size=source_vocab_size, 
                                  encoding_embedding_size=enc_embedding_size,
                                  keep_prob=keep_prob)
    
    
    # Prepare the target sequences we'll feed to the decoder in training mode
    dec_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    
    # Pass encoder state and decoder inputs to the decoders
    training_decoder_output, inference_decoder_output =  decoding_layer(dec_input, enc_state, target_sequence_length, 
                                                                        max_target_sentence_length,
                                                                        rnn_size, num_layers, target_vocab_to_int, 
                                                                        target_vocab_size, batch_size, keep_prob, 
                                                                        dec_embedding_size)
    
    return training_decoder_output, inference_decoder_output


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [19]:
# Number of Epochs
epochs = 10
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 200
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 300
decoding_embedding_size = 300
# Learning Rate
learning_rate = 0.01
# Dropout Keep Probability
keep_probability = 0.8
display_step = 2

### Build the Graph
Build the graph using the neural network you implemented.

In [20]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [21]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    2/1077 - Train Accuracy: 0.0950, Validation Accuracy: 0.0827, Loss: 4.2935
Epoch   0 Batch    4/1077 - Train Accuracy: 0.2684, Validation Accuracy: 0.3420, Loss: 3.5801
Epoch   0 Batch    6/1077 - Train Accuracy: 0.2816, Validation Accuracy: 0.3398, Loss: 3.4192
Epoch   0 Batch    8/1077 - Train Accuracy: 0.3074, Validation Accuracy: 0.3686, Loss: 3.6800
Epoch   0 Batch   10/1077 - Train Accuracy: 0.2718, Validation Accuracy: 0.3626, Loss: 3.4561
Epoch   0 Batch   12/1077 - Train Accuracy: 0.3000, Validation Accuracy: 0.3643, Loss: 3.2810
Epoch   0 Batch   14/1077 - Train Accuracy: 0.3363, Validation Accuracy: 0.3736, Loss: 3.0396
Epoch   0 Batch   16/1077 - Train Accuracy: 0.3383, Validation Accuracy: 0.3821, Loss: 3.0720
Epoch   0 Batch   18/1077 - Train Accuracy: 0.3105, Validation Accuracy: 0.3711, Loss: 2.9888
Epoch   0 Batch   20/1077 - Train Accuracy: 0.3480, Validation Accuracy: 0.3967, Loss: 2.8606
Epoch   0 Batch   22/1077 - Train Accuracy: 0.3566, Validati

Epoch   0 Batch  178/1077 - Train Accuracy: 0.6539, Validation Accuracy: 0.6023, Loss: 0.6909
Epoch   0 Batch  180/1077 - Train Accuracy: 0.6070, Validation Accuracy: 0.5710, Loss: 0.7131
Epoch   0 Batch  182/1077 - Train Accuracy: 0.5885, Validation Accuracy: 0.5391, Loss: 0.7000
Epoch   0 Batch  184/1077 - Train Accuracy: 0.6039, Validation Accuracy: 0.5856, Loss: 0.6652
Epoch   0 Batch  186/1077 - Train Accuracy: 0.5851, Validation Accuracy: 0.5898, Loss: 0.6959
Epoch   0 Batch  188/1077 - Train Accuracy: 0.5816, Validation Accuracy: 0.6090, Loss: 0.6925
Epoch   0 Batch  190/1077 - Train Accuracy: 0.6590, Validation Accuracy: 0.5927, Loss: 0.6538
Epoch   0 Batch  192/1077 - Train Accuracy: 0.6234, Validation Accuracy: 0.5739, Loss: 0.6679
Epoch   0 Batch  194/1077 - Train Accuracy: 0.6276, Validation Accuracy: 0.5742, Loss: 0.6319
Epoch   0 Batch  196/1077 - Train Accuracy: 0.6578, Validation Accuracy: 0.6040, Loss: 0.6383
Epoch   0 Batch  198/1077 - Train Accuracy: 0.6626, Validati

Epoch   0 Batch  354/1077 - Train Accuracy: 0.7422, Validation Accuracy: 0.7418, Loss: 0.4112
Epoch   0 Batch  356/1077 - Train Accuracy: 0.7836, Validation Accuracy: 0.7418, Loss: 0.3686
Epoch   0 Batch  358/1077 - Train Accuracy: 0.7451, Validation Accuracy: 0.7585, Loss: 0.4019
Epoch   0 Batch  360/1077 - Train Accuracy: 0.7656, Validation Accuracy: 0.7347, Loss: 0.3698
Epoch   0 Batch  362/1077 - Train Accuracy: 0.7556, Validation Accuracy: 0.7372, Loss: 0.3669
Epoch   0 Batch  364/1077 - Train Accuracy: 0.7023, Validation Accuracy: 0.7351, Loss: 0.3834
Epoch   0 Batch  366/1077 - Train Accuracy: 0.7266, Validation Accuracy: 0.7376, Loss: 0.3652
Epoch   0 Batch  368/1077 - Train Accuracy: 0.7875, Validation Accuracy: 0.7418, Loss: 0.3688
Epoch   0 Batch  370/1077 - Train Accuracy: 0.8170, Validation Accuracy: 0.7631, Loss: 0.3651
Epoch   0 Batch  372/1077 - Train Accuracy: 0.8027, Validation Accuracy: 0.7440, Loss: 0.3320
Epoch   0 Batch  374/1077 - Train Accuracy: 0.7492, Validati

Epoch   0 Batch  530/1077 - Train Accuracy: 0.8586, Validation Accuracy: 0.8274, Loss: 0.1850
Epoch   0 Batch  532/1077 - Train Accuracy: 0.8664, Validation Accuracy: 0.8434, Loss: 0.2061
Epoch   0 Batch  534/1077 - Train Accuracy: 0.8508, Validation Accuracy: 0.8242, Loss: 0.1809
Epoch   0 Batch  536/1077 - Train Accuracy: 0.8664, Validation Accuracy: 0.8324, Loss: 0.1696
Epoch   0 Batch  538/1077 - Train Accuracy: 0.8843, Validation Accuracy: 0.8207, Loss: 0.1371
Epoch   0 Batch  540/1077 - Train Accuracy: 0.8676, Validation Accuracy: 0.8317, Loss: 0.1542
Epoch   0 Batch  542/1077 - Train Accuracy: 0.8383, Validation Accuracy: 0.8260, Loss: 0.1620
Epoch   0 Batch  544/1077 - Train Accuracy: 0.8746, Validation Accuracy: 0.8374, Loss: 0.1479
Epoch   0 Batch  546/1077 - Train Accuracy: 0.8629, Validation Accuracy: 0.8249, Loss: 0.1714
Epoch   0 Batch  548/1077 - Train Accuracy: 0.8559, Validation Accuracy: 0.8271, Loss: 0.1753
Epoch   0 Batch  550/1077 - Train Accuracy: 0.8418, Validati

Epoch   0 Batch  706/1077 - Train Accuracy: 0.8746, Validation Accuracy: 0.8835, Loss: 0.1414
Epoch   0 Batch  708/1077 - Train Accuracy: 0.8816, Validation Accuracy: 0.8675, Loss: 0.1063
Epoch   0 Batch  710/1077 - Train Accuracy: 0.9102, Validation Accuracy: 0.8839, Loss: 0.0857
Epoch   0 Batch  712/1077 - Train Accuracy: 0.8984, Validation Accuracy: 0.8917, Loss: 0.0956
Epoch   0 Batch  714/1077 - Train Accuracy: 0.8932, Validation Accuracy: 0.8736, Loss: 0.0932
Epoch   0 Batch  716/1077 - Train Accuracy: 0.9012, Validation Accuracy: 0.8892, Loss: 0.0792
Epoch   0 Batch  718/1077 - Train Accuracy: 0.9074, Validation Accuracy: 0.8832, Loss: 0.0954
Epoch   0 Batch  720/1077 - Train Accuracy: 0.9100, Validation Accuracy: 0.8803, Loss: 0.0936
Epoch   0 Batch  722/1077 - Train Accuracy: 0.8980, Validation Accuracy: 0.9077, Loss: 0.0852
Epoch   0 Batch  724/1077 - Train Accuracy: 0.9042, Validation Accuracy: 0.8697, Loss: 0.1021
Epoch   0 Batch  726/1077 - Train Accuracy: 0.9027, Validati

...

Epoch   2 Batch  846/1077 - Train Accuracy: 0.9414, Validation Accuracy: 0.9570, Loss: 0.0541
Epoch   2 Batch  848/1077 - Train Accuracy: 0.9383, Validation Accuracy: 0.9425, Loss: 0.0476
Epoch   2 Batch  850/1077 - Train Accuracy: 0.9263, Validation Accuracy: 0.9524, Loss: 0.0733
Epoch   2 Batch  852/1077 - Train Accuracy: 0.9437, Validation Accuracy: 0.9496, Loss: 0.0582
Epoch   2 Batch  854/1077 - Train Accuracy: 0.9230, Validation Accuracy: 0.9414, Loss: 0.0521
Epoch   2 Batch  856/1077 - Train Accuracy: 0.9363, Validation Accuracy: 0.9517, Loss: 0.0424
Epoch   2 Batch  858/1077 - Train Accuracy: 0.9706, Validation Accuracy: 0.9482, Loss: 0.0276
Epoch   2 Batch  860/1077 - Train Accuracy: 0.9568, Validation Accuracy: 0.9439, Loss: 0.0377
Epoch   2 Batch  862/1077 - Train Accuracy: 0.9355, Validation Accuracy: 0.9382, Loss: 0.0465
Epoch   2 Batch  864/1077 - Train Accuracy: 0.9090, Validation Accuracy: 0.9350, Loss: 0.0392
Epoch   2 Batch  866/1077 - Train Accuracy: 0.9349, Validati

Epoch   2 Batch 1022/1077 - Train Accuracy: 0.9792, Validation Accuracy: 0.9467, Loss: 0.0281
Epoch   2 Batch 1024/1077 - Train Accuracy: 0.9297, Validation Accuracy: 0.9354, Loss: 0.0405
Epoch   2 Batch 1026/1077 - Train Accuracy: 0.9587, Validation Accuracy: 0.9464, Loss: 0.0520
Epoch   2 Batch 1028/1077 - Train Accuracy: 0.9513, Validation Accuracy: 0.9563, Loss: 0.0404
Epoch   2 Batch 1030/1077 - Train Accuracy: 0.9535, Validation Accuracy: 0.9471, Loss: 0.0417
Epoch   2 Batch 1032/1077 - Train Accuracy: 0.9256, Validation Accuracy: 0.9354, Loss: 0.0496
Epoch   2 Batch 1034/1077 - Train Accuracy: 0.9688, Validation Accuracy: 0.9322, Loss: 0.0389
Epoch   2 Batch 1036/1077 - Train Accuracy: 0.9308, Validation Accuracy: 0.9389, Loss: 0.0529
Epoch   2 Batch 1038/1077 - Train Accuracy: 0.9465, Validation Accuracy: 0.9457, Loss: 0.0513
Epoch   2 Batch 1040/1077 - Train Accuracy: 0.9404, Validation Accuracy: 0.9435, Loss: 0.0480
Epoch   2 Batch 1042/1077 - Train Accuracy: 0.9727, Validati

Epoch   3 Batch  124/1077 - Train Accuracy: 0.9457, Validation Accuracy: 0.9244, Loss: 0.0480
Epoch   3 Batch  126/1077 - Train Accuracy: 0.9554, Validation Accuracy: 0.9268, Loss: 0.0395
Epoch   3 Batch  128/1077 - Train Accuracy: 0.9598, Validation Accuracy: 0.9176, Loss: 0.0445
Epoch   3 Batch  130/1077 - Train Accuracy: 0.9535, Validation Accuracy: 0.9080, Loss: 0.0364
Epoch   3 Batch  132/1077 - Train Accuracy: 0.9531, Validation Accuracy: 0.9105, Loss: 0.0450
Epoch   3 Batch  134/1077 - Train Accuracy: 0.9743, Validation Accuracy: 0.9098, Loss: 0.0487
Epoch   3 Batch  136/1077 - Train Accuracy: 0.9516, Validation Accuracy: 0.9094, Loss: 0.0424
Epoch   3 Batch  138/1077 - Train Accuracy: 0.9449, Validation Accuracy: 0.9187, Loss: 0.0328
Epoch   3 Batch  140/1077 - Train Accuracy: 0.9597, Validation Accuracy: 0.9130, Loss: 0.0342
Epoch   3 Batch  142/1077 - Train Accuracy: 0.9647, Validation Accuracy: 0.9137, Loss: 0.0301
Epoch   3 Batch  144/1077 - Train Accuracy: 0.9145, Validati

Epoch   3 Batch  300/1077 - Train Accuracy: 0.9482, Validation Accuracy: 0.9180, Loss: 0.0332
Epoch   3 Batch  302/1077 - Train Accuracy: 0.9512, Validation Accuracy: 0.9165, Loss: 0.0328
Epoch   3 Batch  304/1077 - Train Accuracy: 0.9539, Validation Accuracy: 0.9190, Loss: 0.0371
Epoch   3 Batch  306/1077 - Train Accuracy: 0.9531, Validation Accuracy: 0.9155, Loss: 0.0506
Epoch   3 Batch  308/1077 - Train Accuracy: 0.9219, Validation Accuracy: 0.9450, Loss: 0.0489
Epoch   3 Batch  310/1077 - Train Accuracy: 0.9520, Validation Accuracy: 0.9517, Loss: 0.0403
Epoch   3 Batch  312/1077 - Train Accuracy: 0.9453, Validation Accuracy: 0.9517, Loss: 0.0526
Epoch   3 Batch  314/1077 - Train Accuracy: 0.9262, Validation Accuracy: 0.9453, Loss: 0.0459
Epoch   3 Batch  316/1077 - Train Accuracy: 0.9609, Validation Accuracy: 0.9304, Loss: 0.0384
Epoch   3 Batch  318/1077 - Train Accuracy: 0.9719, Validation Accuracy: 0.9414, Loss: 0.0277
Epoch   3 Batch  320/1077 - Train Accuracy: 0.9488, Validati

Epoch   3 Batch  476/1077 - Train Accuracy: 0.9613, Validation Accuracy: 0.9201, Loss: 0.0357
Epoch   3 Batch  478/1077 - Train Accuracy: 0.9367, Validation Accuracy: 0.9286, Loss: 0.0361
Epoch   3 Batch  480/1077 - Train Accuracy: 0.9330, Validation Accuracy: 0.9205, Loss: 0.0365
Epoch   3 Batch  482/1077 - Train Accuracy: 0.9264, Validation Accuracy: 0.9205, Loss: 0.0468
Epoch   3 Batch  484/1077 - Train Accuracy: 0.9563, Validation Accuracy: 0.9183, Loss: 0.0495
Epoch   3 Batch  486/1077 - Train Accuracy: 0.9539, Validation Accuracy: 0.9254, Loss: 0.0348
Epoch   3 Batch  488/1077 - Train Accuracy: 0.9363, Validation Accuracy: 0.9354, Loss: 0.0395
Epoch   3 Batch  490/1077 - Train Accuracy: 0.9191, Validation Accuracy: 0.9339, Loss: 0.0470
Epoch   3 Batch  492/1077 - Train Accuracy: 0.9621, Validation Accuracy: 0.9297, Loss: 0.0449
Epoch   3 Batch  494/1077 - Train Accuracy: 0.9395, Validation Accuracy: 0.9272, Loss: 0.0312
Epoch   3 Batch  496/1077 - Train Accuracy: 0.9563, Validati

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [23]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [24]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [28]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    lower = sentence.lower()
    words =lower.split()
    
    return [vocab_to_int[w] if w in vocab_to_int else vocab_to_int['<UNK>'] for w in words]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [29]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [134, 221, 88, 158, 193, 13, 123]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [257, 32, 107, 168, 192, 85, 284, 132, 1]
  French Words: il a vu un nouveau camion jaune . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.